# Example expected syntax of the R client
Not expected to be fully functional yet

In [1]:
library(devtools)
options(unzip = 'internal'); install_github("Open-EO/openeo-r-backend", ref="develop") # Currently takes a while
install_github("Open-EO/openeo-r-client", ref="develop")
library(openeo)

Skipping install of 'openEO.R.Backend' from a github remote, the SHA1 (dfc7d685) has not changed since last install.
  Use `force = TRUE` to force installation
Skipping install of 'openeo' from a github remote, the SHA1 (e6bf72e5) has not changed since last install.
  Use `force = TRUE` to force installation
Loading required package: magrittr
Loading required package: jsonlite
Loading required package: gdalUtils


(Potentially there should be an ability to explore available/connected backends)

## Connect to a backend

In [4]:
conn = connect("http://localhost:8000/api", "test", "test", rbackend=TRUE)

Registered 'http://localhost:8000/api' as host
Login successful.

## Listing backend capabilities:
Get a list of functions that the backend natively supports; names and descriptions (list/named vector)

In [5]:
listProcesses(conn)

process_id,description
filter_daterange,filters a data set with a temporal dimension based on a stated start and end date
find_min,calculates the minimum value per pixel of a single valued band collection
calculate_ndvi,Calculates the ndvi per pixel and scene in a given collection


Same for listing collections on the backend

In [6]:
listCollections(conn)

product_id,description,source
landsat7_ndvi,Landsat7 NDVI calculation,Marius Appel
sentinel2_subset,Sentinel 2 raster time series for a small spatial subset,ESA / Marius Appel


Get in-depth information about a process or collection

In [11]:
describe(conn, "filter_daterange")

[[1]]
[[1]]$process_id
[1] "filter_daterange"

[[1]]$description
[1] "filters a data set with a temporal dimension based on a stated start and end date"

[[1]]$args
[[1]]$args$imagery
[[1]]$args$imagery$description
[1] "the temporal dataset/collection"


[[1]]$args$from
[[1]]$args$from$description
[1] "start date/timestamp for the query interval"


[[1]]$args$to
[[1]]$args$to$description
[1] "end date/timestamp for the query interval"

In [21]:
describe(conn, product_id=c("sentinel2_subset", "landsat7_ndvi"))

ERROR while rich displaying an object: Error in vapply(seq_along(mapped), function(i) {: values must be length 1,
 but FUN(X[[2]]) result is length 0

Traceback:
1. FUN(X[[i]], ...)
2. tryCatch(withCallingHandlers({
 .     rpr <- mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler), error = outer_handler)
3. tryCatchList(expr, classes, parentenv, handlers)
4. tryCatchOne(expr, names, parentenv, handlers[[1L]])
5. doTryCatch(return(expr), name, parentenv, handler)
6. withCallingHandlers({
 .     rpr <- mime2repr[[mime]](obj)
 .     if (is.null(rpr)) 
 .         return(NULL)
 .     prepare_content(is.raw(rpr), rpr)
 . }, error = error_handler)
7. mime2repr[[mime]](obj)
8. repr_html.list(obj)
9. repr_list_generic(obj, "html", "\t<li>%s</li>\n", "\t<dt>$%s</dt>\n\t\t<dd>%s</dd>\n", 
 .     "<strong>$%s</strong> = %s", "<ol>\n%s</ol>\n", "<dl>\n%s</dl>\n", 
 .     numeric_item = "\t<dt>[[%s]]</dt>\n\t\

[[1]]
[[1]]$product_id
[1] "sentinel2_subset"

[[1]]$description
[1] "Sentinel 2 raster time series for a small spatial subset"

[[1]]$source
[1] "ESA / Marius Appel"

[[1]]$extent
class       : Extent 
xmin        : 699960 
xmax        : 702960 
ymin        : 7897000 
ymax        : 7900000 

[[1]]$time
[[1]]$time$from
[1] "2016-12-12 08:23:32 UTC"

[[1]]$time$to
[1] "2017-08-29 08:16:01 UTC"


[[1]]$bands
[[1]]$bands[[1]]
[[1]]$bands[[1]]$band_id
[1] "1"

[[1]]$bands[[1]]$name
NULL

[[1]]$bands[[1]]$wavelength_nm
[1] "443"

[[1]]$bands[[1]]$res
[[1]]$bands[[1]]$res$x
[1] 60

[[1]]$bands[[1]]$res$y
[1] 60


[[1]]$bands[[1]]$scale
[1] 1

[[1]]$bands[[1]]$offset
[1] 0

[[1]]$bands[[1]]$type
[1] "uint16"

[[1]]$bands[[1]]$unit
NULL

[[1]]$bands[[1]]$nodata
[1] 0


[[1]]$bands[[2]]
[[1]]$bands[[2]]$band_id
[1] "2"

[[1]]$bands[[2]]$name
[1] "blue"

[[1]]$bands[[2]]$wavelength_nm
[1] "490"

[[1]]$bands[[2]]$res
[[1]]$bands[[2]]$res$x
[1] 10

[[1]]$bands[[2]]$res$y
[1] 10


[[1]]$bands[[2]]$

Listing existing user jobs

In [9]:
listJobs(conn)

ERROR: Error in private$GET(endpoint, authorized = TRUE, type = "application/json"): Cannot access data endpoint


Listing which UDF capabilities/packages are supported

In [10]:
listUDFCapabilities(conn)

ERROR: Error in eval(expr, envir, enclos): could not find function "listUDFCapabilities"


## Simple task: calculating NDVI using a linked list of processes
Assumes NDVI function takes band names as parameters

In [12]:
Task = collection("sentinel2_subset") %>% process("filter_daterange", from="2016", to="2018") %>%
    process("calculate_ndvi", red="B04", nir="B8A")
str(Task)
taskToJSON(Task)

List of 2
 $ process_id: chr "calculate_ndvi"
 $ args      :List of 3
  ..$ collections:List of 2
  .. ..$ process_id: chr "filter_daterange"
  .. ..$ args      :List of 3
  .. .. ..$ collections:List of 1
  .. .. .. ..$ collection_id: chr "sentinel2_subset"
  .. .. ..$ from       : chr "2016"
  .. .. ..$ to         : chr "2018"
  ..$ red        : chr "B04"
  ..$ nir        : chr "B8A"


{
  "process_id": "calculate_ndvi",
  "args": {
    "collections": {
      "process_id": "filter_daterange",
      "args": {
        "collections": {
          "collection_id": "sentinel2_subset"
        },
        "from": "2016",
        "to": "2018"
      }
    },
    "red": "B04",
    "nir": "B8A"
  }
} 

## Advanced task: calculating NDVI using processes that have other processes as arguments 
Assumes NDVI function takes process graphs

In [10]:
RedSubset = collection("S2_L2A_T32TPS_20M") %>% process("filter_band", band="red")
NIRSubset = collection("S2_L2A_T32TPS_20M") %>% process("filter_band", band="nir")
Task = collection("S2_L2A_T32TPS_20M") %>% process("filter_daterange", start="2016", end="2018") %>%
    process("NDVI", red=RedSubset, nir=NIRSubset)
str(Task)
taskToJSON(Task)

List of 2
 $ process_id: chr "NDVI"
 $ args      :List of 3
  ..$ collections:List of 2
  .. ..$ process_id: chr "filter_daterange"
  .. ..$ args      :List of 3
  .. .. ..$ collections:List of 1
  .. .. .. ..$ collection_id: chr "S2_L2A_T32TPS_20M"
  .. .. ..$ start      : chr "2016"
  .. .. ..$ end        : chr "2018"
  ..$ red        :List of 2
  .. ..$ process_id: chr "filter_band"
  .. ..$ args      :List of 2
  .. .. ..$ collections:List of 1
  .. .. .. ..$ collection_id: chr "S2_L2A_T32TPS_20M"
  .. .. ..$ band       : chr "red"
  ..$ nir        :List of 2
  .. ..$ process_id: chr "filter_band"
  .. ..$ args      :List of 2
  .. .. ..$ collections:List of 1
  .. .. .. ..$ collection_id: chr "S2_L2A_T32TPS_20M"
  .. .. ..$ band       : chr "nir"


{
  "process_id": "NDVI",
  "args": {
    "collections": {
      "process_id": "filter_daterange",
      "args": {
        "collections": {
          "collection_id": "S2_L2A_T32TPS_20M"
        },
        "start": "2016",
        "end": "2018"
      }
    },
    "red": {
      "process_id": "filter_band",
      "args": {
        "collections": {
          "collection_id": "S2_L2A_T32TPS_20M"
        },
        "band": "red"
      }
    },
    "nir": {
      "process_id": "filter_band",
      "args": {
        "collections": {
          "collection_id": "S2_L2A_T32TPS_20M"
        },
        "band": "nir"
      }
    }
  }
} 

## Three ways of processing data in OpenEO:
### Synchronous
Run right away and give data as an object (Raster?); 'format' optional, should default to native that is then read into a Raster* object

In [13]:
Result = executeTask(conn, Task, format="GTiff")

No encoding supplied: defaulting to UTF-8.


ERROR: Error in con$execute(task, format, output): {"status":400,"message":"Missing query parameter \"evaluate\" or it contains a value other then \"lazy\" or \"batch\""}


### Lazy
Ask the server to run it when needed (on WCS, but also download etc.)

In [18]:
(JobID = queueTask(conn, Task))

NULL

### Batch
Ask the server to prepare it the USGS ESPA way, get a job ID and the URL to the output location. This is actually a lazy job followed by a download link request.

In [14]:
OutputInfo = order(conn, Task)
(JobID = OutputInfo$JobID)

Warning message in is.na(x):
“is.na() applied to non-(list or vector) of type 'environment'”

ERROR: Error in x[!nas]: object of type 'environment' is not subsettable


## Functions that can be performed with a JobID
Lazy (and batch) only

In [17]:
followJob(conn, JobID) # cat --follow style updates about the job
(JobInfo = queryJob(conn, JobID)) # Get current information on a job
(Result = downloadJob(conn, JobID)) # Get the result as with the synchronous case
(WCSURL = getWCSLink(conn, JobID)) # Get a URL to the WCS to visualise/download data
cancelJob(conn, JobID) # Ask to delete a job, the backend eventually removes it

NULL

NULL

NULL

NULL

ERROR: Error in eval(expr, envir, enclos): could not find function "getWCSLink"


## Either way have a result, should be a **Raster*** object

In [ ]:
plot(Result)
spplot(Result, 3)

## Theoretical use case 1: composite example

In [22]:
CompositeTask = collection("S2_L2A_T32TPS_20M") %>% process("date_range_filter", start="2016-01-01", end="2016-03-10") %>%
    process("bbox_filter", left=652000, right=672000, top=5161000, bottom=5181000, srs="EPSG:32632") %>%
    process("max_time")

str(CompositeTask)
taskToJSON(CompositeTask)

List of 2
 $ process_id: chr "max_time"
 $ args      :List of 1
  ..$ collections:List of 2
  .. ..$ process_id: chr "bbox_filter"
  .. ..$ args      :List of 6
  .. .. ..$ collections:List of 2
  .. .. .. ..$ process_id: chr "date_range_filter"
  .. .. .. ..$ args      :List of 3
  .. .. .. .. ..$ collections:List of 1
  .. .. .. .. .. ..$ collection_id: chr "S2_L2A_T32TPS_20M"
  .. .. .. .. ..$ start      : chr "2016-01-01"
  .. .. .. .. ..$ end        : chr "2016-03-10"
  .. .. ..$ left       : num 652000
  .. .. ..$ right      : num 672000
  .. .. ..$ top        : num 5161000
  .. .. ..$ bottom     : num 5181000
  .. .. ..$ srs        : chr "EPSG:32632"


{
  "process_id": "max_time",
  "args": {
    "collections": {
      "process_id": "bbox_filter",
      "args": {
        "collections": {
          "process_id": "date_range_filter",
          "args": {
            "collections": {
              "collection_id": "S2_L2A_T32TPS_20M"
            },
            "start": "2016-01-01",
            "end": "2016-03-10"
          }
        },
        "left": 652000,
        "right": 672000,
        "top": 5161000,
        "bottom": 5181000,
        "srs": "EPSG:32632"
      }
    }
  }
} 

### Lazy

In [15]:
Conn = connect(host="http://saocompute.eurac.edu/openEO_WCPS_Driver", user="nobody", password="nobody")
Job = queueTask(Conn, CompositeTask)
OutPath = file.path("Downloads", "Result.netcdf")
downloadJob(Conn, Job, OutPath, "netcdf") # Processing happens here
Result = brick(OutPath)

Registered 'http://saocompute.eurac.edu/openEO_WCPS_Driver' as host


ERROR: Error in OpenEOClient$new()$connect(url = host)$login(user = user, password = password): Login failed.


### Synchronous

In [ ]:
Result = executeTask(Conn, CompositeTask)

In [ ]:
plot(Result)

## Complex use case: Land cover classification

UDF definitions: uploading a file so that it can be used as a process

In [19]:
defineUDF(conn, language="R", target="resample", type="aggregate_space", content=file.path("resample.r")) # This could be a server-defined function too
defineUDF(conn, language="R", target="temporal_cloud_filter", type="apply_time", content=file.path("temporal_cloud_filter.r")) # No such type in API yet
defineUDF(conn, language="R", target="harmonic_analysis", type="reduce_time", content=file.path("harmonic_analysis.r"))
defineUDF(conn, language="R", target="ranger_classification", type="apply_pixel", content=file.path("ranger_classification.r"))

$process_id
[1] "/udf/R/aggregate_space"

$args
$args$script
[1] "resample"

$process_id
[1] "/udf/R/apply_time"

$args
$args$script
[1] "temporal_cloud_filter"

$process_id
[1] "/udf/R/reduce_time"

$args
$args$script
[1] "harmonic_analysis"

$process_id
[1] "/udf/R/apply_pixel"

$args
$args$script
[1] "ranger_classification"

Process graphs

In [21]:
DEMCollection = collection("GLSDEM") %>%
    process("bbox_filter", left=652000, right=672000, top=5161000, bottom=5181000, srs="EPSG:32632") %>%
    process("resample", input=collection("S2_L2A_T32TPS_20M"))
    # UDF to resample to Sentinel 2 pixel size

CloudlessNDVI = collection("S2_L2A_T32TPS_20M") %>% # Graph part reused in several places
    process("bbox_filter", left=652000, right=672000, top=5161000, bottom=5181000, srs="EPSG:32632") %>%
    process("mask", mask="CM", maskvalue=5) %>% process("ndvi", red="B04", nir="B8A") %>%
    process("temporal_cloud_filter", band="B02", threshold="50")
    # Temporal cloud filter UDF: for each pixel use blue to detect outliers, replace more than 50 with NA

TSParams = CloudlessNDVI %>% process("harmonic_analysis")
# Harmonic parameter extraction UDF: get several layers of harmonic model params

MeanNDVI = CloudlessNDVI %>% process("mean_time")
# Also get the mean NDVI as a covariate

# Somehow upload training dataset or have a UDF that downloads it(?)
LandCoverTask = collection(NULL) %>% process("ranger_classification", NDVI=MeanNDVI, DEM=DEMCollection, TS=TSParams)
# UDF: runs a ranger training and prediction pass, returns an LC map and some (spatial?) statistics

str(LandCoverTask)
taskToJSON(LandCoverTask)

List of 2
 $ process_id: chr "ranger_classification"
 $ args      :List of 4
  ..$ collections:List of 1
  .. ..$ collection_id: NULL
  ..$ NDVI       :List of 2
  .. ..$ process_id: chr "mean_time"
  .. ..$ args      :List of 1
  .. .. ..$ collections:List of 2
  .. .. .. ..$ process_id: chr "temporal_cloud_filter"
  .. .. .. ..$ args      :List of 3
  .. .. .. .. ..$ collections:List of 2
  .. .. .. .. .. ..$ process_id: chr "ndvi"
  .. .. .. .. .. ..$ args      :List of 3
  .. .. .. .. .. .. ..$ collections:List of 2
  .. .. .. .. .. .. .. ..$ process_id: chr "mask"
  .. .. .. .. .. .. .. ..$ args      :List of 3
  .. .. .. .. .. .. .. .. ..$ collections:List of 2
  .. .. .. .. .. .. .. .. .. ..$ process_id: chr "bbox_filter"
  .. .. .. .. .. .. .. .. .. ..$ args      :List of 6
  .. .. .. .. .. .. .. .. .. .. ..$ collections:List of 1
  .. .. .. .. .. .. .. .. .. .. .. ..$ collection_id: chr "S2_L2A_T32TPS_20M"
  .. .. .. .. .. .. .. .. .. .. ..$ left       : num 652000
  .. .. .. 

{
  "process_id": "ranger_classification",
  "args": {
    "collections": {
      "collection_id": {}
    },
    "NDVI": {
      "process_id": "mean_time",
      "args": {
        "collections": {
          "process_id": "temporal_cloud_filter",
          "args": {
            "collections": {
              "process_id": "ndvi",
              "args": {
                "collections": {
                  "process_id": "mask",
                  "args": {
                    "collections": {
                      "process_id": "bbox_filter",
                      "args": {
                        "collections": {
                          "collection_id": "S2_L2A_T32TPS_20M"
                        },
                        "left": 652000,
                        "right": 672000,
                        "top": 5161000,
                        "bottom": 5181000,
                        "srs": "EPSG:32632"
                      }
                    },
                    "mask": "CM",
    

In [ ]:
Result = executeTask(conn, LandCoverTask)
spplot(Result, 1)